In [ ]:
import os 

from multiprocessing import Pool, cpu_count

import numpy as np

from astropy.nddata import CCDData, Cutout2D
from astropy.io import fits
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy import units as u

from photutils import source_properties, SourceProperties

In [ ]:
from petrofit.segmentation import make_segments, deblend_segments, plot_segments
from petrofit.petrosian import Petrosian, PetrosianCorrection
from petrofit.photometry import get_source_position, object_photometry, flux_to_abmag
from petrofit.utils import angular_to_pixel

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import cm

plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['image.origin'] = 'lower'

In [ ]:
import ipywidgets as widgets
from IPython.display import display

DISPLAY_STYLE = {'description_width': 'initial'}

# Define Paths

In this section we define the paths to the Hubble Space Telescope Frontier Fields. The fields are located on the [STScI data archive website]( https://archive.stsci.edu/pub/hlsp/frontier). There are 6 clusters and each cluster has 7 band images (3 ACS and 4 WFC3). Here we generate and store the URL paths to different band images of a single cluster so we can download them.

In [ ]:
acs_filter_list = ['f435w', 'f606w', 'f814w']
wfc3_filter_list = ['f105w', 'f125w', 'f140w', 'f160w']

input_data_formatter = "https://archive.stsci.edu/pub/hlsp/frontier/abell2744/images/hst/v1.0/hlsp_frontier_hst_{}-60mas_abell2744_{}_v1.0_drz.fits"

In [ ]:
program_name = os.path.basename(input_data_formatter).replace("_{}","").replace('-','')
program_name = os.path.splitext(program_name)[0]
program_name

In [ ]:
data_paths = {}
for f in acs_filter_list:
    data_paths[f] = input_data_formatter.format('acs', f)
    
for f in wfc3_filter_list:
    data_paths[f] = input_data_formatter.format('wfc3', f)
    
petrosian_cat_collection = {}

In [ ]:
for f in acs_filter_list + wfc3_filter_list:
    print("{}: {}".format(f, data_paths[f]))

# User Input

We select the center of the cluster in this section and the size of the cutout that will be analyzed. We also specify the center and cutout size of a patch of the image that we will use to estimate the background noise stats.

In [ ]:
show_plots = True

# Dark patch:
# A dark patch in the image to measure background stats
noise_cutout_center = (2760, 3420)
noise_cutout_size = 70

# Center of cluster:
cx, cy = [2412, 2519]
data_cutout_size = 2000

# Detection Image

Before we can analyze images of galaxies, we first need to make a catalog of their positions. To locate galaxies, we first select a band of interest and download the image:

In [ ]:
detection_filter = 'f105w'
input_data_path = data_paths[detection_filter]
detection_data = CCDData.read(input_data_path, cache=True)

### Estimate detection data noise at dark area

The following cell makes a cutout of the noise image (cutout of dark patch we specified) and measure the mean, std and sigma values for later use.

In [ ]:
detection_noise_cutout = Cutout2D(detection_data, noise_cutout_center, noise_cutout_size)

detection_noise_mean = detection_noise_cutout.data.mean()
detection_noise_sigma = detection_noise_cutout.data.std()
detection_noise_3_sigma = detection_noise_sigma * 3.
detection_noise_8_sigma = detection_noise_sigma * 8.

print(detection_noise_mean, detection_noise_3_sigma, detection_noise_8_sigma)
    
if show_plots:
    plt.imshow(detection_noise_cutout.data, 
               vmax=detection_noise_mean+detection_noise_3_sigma, 
               vmin=detection_noise_mean-detection_noise_3_sigma)
    plt.xlabel('Pix')
    plt.ylabel('pix')
    plt.show()
    
    n, bins, patches = plt.hist(detection_noise_cutout.data.flatten(), bins=35, align='left', color='black')
    plt.plot(bins[:-1], n, c='r', linewidth=3)

    plt.xlabel('Flux Bins [{}]'.format(str(detection_data.unit)))
    plt.ylabel('Count')
    plt.title('Noise Histogram')
    plt.show()

### Cutout Detection Image

In this section we make a cutout of the cluster at the user specified center and size. We also compute some stats that will become useful in the detection stage. We also use the detection image std to set the `vmax` and `vmin` of the detection image plots.

In [ ]:
detection_image = Cutout2D(detection_data, (cx, cy), data_cutout_size, copy=True)

# Compute image stats
detection_image_min = detection_image.data.min()
detection_image_max = detection_image.data.max()
detection_image_mean = detection_image.data.mean()
detection_image_sigma = detection_image.data.std()
detection_image_3_sigma = detection_image_sigma * 3.
detection_image_8_sigma = detection_image_sigma * 8.

detection_image_clipped_mean, detection_image_clipped_median, detection_image_clipped_std = sigma_clipped_stats(detection_image.data, sigma=3.0)

# Subtract Noise
# detection_image.data  -= detection_image_clipped_mean
# detection_image.data = np.clip(detection_image.data, -detection_image_3_sigma, np.inf)

# Set min and max values for all plots
vmin = -10 * detection_image_clipped_std
vmax = +10 * detection_image_clipped_std

if show_plots:
    plt.imshow(detection_image.data, vmin=vmin, vmax=vmax)
    plt.xlabel('Pix')
    plt.ylabel('pix')
    plt.show()

#print(image_mean)

### Segmentation and Main Catalog

In this section we use PhotUtils to make a segmentation map and deblend the identified sources. Please note that `make_segments` and `deblend_segments` are wrappers for their respective PhotUtils functions that make it easy to pass arguments. 

In [ ]:
# Define detect threshold
nsigma = np.ones_like(detection_image.data)
nsigma[:, :] = detection_noise_3_sigma

# Define smoothing kernel
kernel_size = 5
fwhm = 5
npixels = 5**2

# Make segmentation map
detection_segm = make_segments(detection_image.data, nsigma=nsigma, kernel_size=kernel_size, fwhm=fwhm, npixels=npixels)

# Deblend segmentation map
detection_segm_deblend = deblend_segments(detection_image.data, detection_segm, contrast=0, nlevels=50,
                                         kernel_size=kernel_size, fwhm=fwhm, npixels=npixels)

# Remove border sources
#detection_segm_deblend.remove_border_labels(200)

# Make catalog
detection_cat = source_properties(detection_image.data, detection_segm_deblend, wcs=detection_image.wcs)

# Display source properties
print("Num of Targets:", len(detection_cat))

# Plot segments
if show_plots:
    plot_segments(detection_segm_deblend, detection_image.data, vmin=vmin, vmax=vmax)
    plt.show()

# Measure Petrosian

We measure here the Petrosian radii of sources by analyzing their radial light profiles. To do this we select the max pixel to be equivalent to 15 arcsec and sample at every pixel up to the max pixel.

In [ ]:
# Define Radii
# ------------
# Calculate galaxy size
galaxy_pixel_size = angular_to_pixel(15 * u.arcsec, wcs=detection_image.wcs) # convert to pixels

# Define max rad in pixels and number of apertures
max_pix = int(round(galaxy_pixel_size))
n = int(max_pix)

# Create list of radii
r_list = [x * int(max_pix / n) for x in range(1, n+1)]
r_list = np.array(r_list)

print("galaxy_pixel_size = {:0.2f} pix".format(galaxy_pixel_size))
print("max_pix = {} pix".format(max_pix, n))
print("n = {}".format(max_pix, n))
print("len(r_list) = {}".format(len(r_list)))

## Detection Photometry and Petrosian Calculation 

For each source in the detection catalog, measure the Petrosian properties. Before making the measurement, we measure the photometery of the objects by :
1. Estimate a good size for the cutout around the target source (`cutout_size`) using the size of the segmentation box.
2. Clip the radius array if it is larger than the cutout image (to save compute time).
3. Mask all sources that are not the target.
4. Make a cutout of the target.
5. To subtract the background around the source, we fit a 2D plane to the cutout pixels that are below the `sigma` parameter. We then turn the fitted 2D plane model into an image, which we subtract from the cutout. 
6.  We measure the photometry at each radius in the clipped radius list. The shape of the apertures is elliptical. The elongation and orientation of the ellipse are determined by the of the source in the segmentation output.
7. We then use the output of the photometry to construct a curve of growth and construct a  `Petrosian` object which can be used to infer Petrosian properties (radius, C2080 etc..)

In [ ]:
pb = widgets.IntProgress(
    value=0,
    min=0,
    max=len(detection_cat),
    step=1,
    description='Loading:',
    bar_style='',
    orientation='horizontal',
    style=DISPLAY_STYLE,
)
display(pb)

petrosian_properties_rows = []

for idx, obj in enumerate(detection_cat):
    pb.value = idx + 1
    pb.description = "{}: {}/{}".format(detection_filter, pb.value, len(detection_cat))
    
    bbox = max(obj.bbox.ixmax - obj.bbox.ixmin, obj.bbox.iymax - obj.bbox.iymin)
    max_r = bbox * 3
    cutout_size = bbox * 3
    
    flux_arr, area_arr, error_arr = object_photometry(
        obj, detection_image.data, 
        detection_segm_deblend, r_list[:max_r], 
        plot=False, sigma=1, sigma_type='clip',
        vmax=vmax/10, vmin=0, 
        bkg_sub=True, mask_background=False,
        cutout_size=cutout_size, #method='center'
    )
    plt.show()
    
    p = Petrosian(r_list[:max_r], area_arr[:max_r], flux_arr[:max_r],)
    petrosian_properties_rows.append(p)
    
    if False:
        p.plot(plot_r=False, plot_normalized_flux=False)
        plt.show()

# Photometry

In [ ]:
def photometry_process_image(current_filter, show_plots=False):
    print(current_filter + " ")

    input_data_path = data_paths[current_filter]
    output_path = "{}_petrosian_catalog.csv".format(os.path.splitext(os.path.basename(input_data_path))[0])
    
    # Load data and vitals
    # ---------------------
    #print('Load data and vitals')
    data = CCDData.read(input_data_path)
    rms_data = fits.getdata(input_data_path.replace('drz', 'rms'))
    
    if show_plots:
        plt.imshow(data, vmin=0, vmax=data.data.mean()*10)
        plt.show()
        
    # Estimate data noise at dark area
    # --------------------------------
    #print('Estimate data noise at dark area')
    noise_cutout = Cutout2D(data, noise_cutout_center, noise_cutout_size)

    noise_mean = noise_cutout.data.mean()
    noise_sigma = noise_cutout.data.std()
    noise_3_sigma = noise_sigma * 3.
    noise_8_sigma = noise_sigma * 8.

    #print(noise_mean, noise_3_sigma, noise_8_sigma)
    
    if show_plots:
        plt.imshow(noise_cutout.data, vmax=noise_mean+noise_3_sigma, vmin=noise_mean-noise_3_sigma)
        plt.show()
    
        n, bins, patches = plt.hist(noise_cutout.data.flatten(), bins=35, align='left', color='black')
        plt.plot(bins[:-1], n, c='r', linewidth=3)

        plt.xlabel('Flux Bins [{}]'.format(str(data.unit)))
        plt.ylabel('Count')
        plt.title('Noise Histogram')
        plt.show()
    
    # Cutout Image
    # -------------
    image = Cutout2D(data, (cx, cy), data_cutout_size, copy=True)
    rms_image = Cutout2D(rms_data, (cx, cy), data_cutout_size, copy=True)

    # Compute image stats
    image_min = image.data.min()
    image_max = image.data.max()
    image_mean = image.data.mean()
    image_sigma = image.data.std()
    image_3_sigma = image_sigma * 3.
    image_8_sigma = image_sigma * 8.

    image_clipped_mean, image_clipped_median, image_clipped_std = sigma_clipped_stats(image.data, sigma=3.0)

    # Subtract Noise
    #image.data  -= noise_mean

    # Set min and max values for all plots
    vmin = -10 * image_clipped_std
    vmax = +10 * image_clipped_std
    
    if show_plots:
        plt.imshow(image.data, vmin=vmin, vmax=vmax)
        plt.show()

    #print(image_mean)
    
    # Segmentation
    # ------------
    
    # Make segmentation map
    segm = detection_cat

    # Deblend segmentation map
    segm_deblend = detection_segm_deblend

    # Remove border sources
    #segm_deblend.remove_border_labels(1)

    # Make catalog
    cat = detection_cat

    # Display source properties
    #print("Num of Targets:", len(cat))

    # Plot segments
    if show_plots:
        plot_segments(segm_deblend, image.data, vmin=vmin, vmax=vmax)
        plt.show()
        
    # Photomerty
    # ----------
    pb = widgets.IntProgress(
        value=0,
        min=0,
        max=len(cat),
        step=1,
        description='Loading:',
        bar_style='',
        orientation='horizontal',
        style=DISPLAY_STYLE,
        
    )
    display(pb)

    photometry_rows = []
    error_rows = []
    mag_rows = []
    mag_error_rows = []
    for idx, obj in enumerate(cat):
        pb.value = idx + 1
        pb.description = "{}: {}/{}".format(current_filter, pb.value, len(cat))
        
        petrosian_properties = petrosian_properties_rows[idx]
        
        r_total_flux = petrosian_properties.r_total_flux
        r_half_light = petrosian_properties.r_half_light
        if np.any(np.isnan(np.array([r_half_light, r_total_flux]))):
            nan_values = np.array([np.nan, np.nan])
            photometry_rows.append(nan_values)
            error_rows.append(nan_values)
            mag_rows.append(nan_values)
            mag_error_rows.append(nan_values)
            continue
            
        bbox = max(obj.bbox.ixmax - obj.bbox.ixmin, obj.bbox.iymax - obj.bbox.iymin)
        cutout_size = max(obj.bbox.ixmax - obj.bbox.ixmin, obj.bbox.iymax - obj.bbox.iymin)#r_total_flux * 2
        cutout_size = r_total_flux * 2.5
         
        flux_arr, area_arr, error_arr = object_photometry(
            obj, image.data, 
            segm_deblend, [r_half_light, r_total_flux],
            error=rms_image.data,
            plot=show_plots, sigma=1, vmax=vmax, vmin=vmin, 
            bkg_sub=True, mask_background=False, 
            cutout_size=cutout_size
        )
        
        #error_arr = np.sqrt(flux_arr)
        
        photometry_rows.append(flux_arr)
        error_rows.append(error_arr)
        mag_rows.append(flux_to_abmag(flux_arr, data.header))
        mag_error_rows.append(flux_to_abmag(flux_arr, data.header))


    return photometry_rows, error_rows, mag_rows, mag_error_rows

In [ ]:
# multiprocessing 
if __name__ == '__main__':
    with Pool(processes=cpu_count()) as pool: 
        print("Starting MP")
        try:
            photometry_results = pool.map(photometry_process_image, acs_filter_list + wfc3_filter_list)
        except Exception as e:
            pool.close()
            raise e 
            
    print("Done")

# Combine Catalogs

In [ ]:
columns = []
column_names = []

r_petrosian_list = []
r_half_light_list = []
r_half_light_arcsec_list = []
r_total_flux_list = []
r_total_flux_arcsec_list = []
r20_list = []
r80_list = []
c2080_list = []

for p in petrosian_properties_rows:
    r_petrosian_list.append(p.r_petrosian)
    r_half_light_list.append(p.r_half_light)
    r_half_light_arcsec_list.append(p.r_half_light_arcsec)
    r_total_flux_list.append(p.r_total_flux)
    r_total_flux_arcsec_list.append(p.r_total_flux_arcsec)
    r20_list.append(p.concentration_index()[0])
    r80_list.append(p.concentration_index()[1])
    c2080_list .append(p.concentration_index()[2])    
    
columns.append(np.array(r_petrosian_list))
column_names.append('r_petrosian')

columns.append(np.array(r_half_light_list))
column_names.append('r_half_light')

columns.append(np.array(r_half_light_arcsec_list))
column_names.append('r_half_light_arcsec')

columns.append(np.array(r_total_flux_list))
column_names.append('r_total_flux')

columns.append(np.array(r_total_flux_arcsec_list))
column_names.append('r_total_flux_arcsec')

columns.append(np.array(r20_list))
column_names.append('r20')

columns.append(np.array(r80_list))
column_names.append('r80')

columns.append(np.array(c2080_list))
column_names.append('c2080')

for idx,  filt in  enumerate(acs_filter_list + wfc3_filter_list):
    print(filt)
    f1, f2 = np.transpose(np.array(photometry_results[idx][0]))
    err1, err2 = np.transpose(np.array(photometry_results[idx][1]))
    m1, m2 = np.transpose(np.array(photometry_results[idx][2]))
    merr1, merr2 = np.transpose(np.array(photometry_results[idx][3]))
    
    # Plot 
    total_hl_diff = m2 - m1 + 2.5*np.log10(2)
    
    plt.scatter(m2, total_hl_diff, label=filt)
    plt.title(' m_total - "double" m_half_light')
    plt.xlabel("m_total")
    plt.ylabel("m_total - m_half_light + 2.5 * log_10(2)")
    
    columns.append(f1)
    column_names.append('f_half_light_{}'.format(filt))
    
    columns.append(err1)
    column_names.append('f_half_light_err_{}'.format(filt))
    
    columns.append(f2)
    column_names.append('f_total_{}'.format(filt))

    columns.append(err2)
    column_names.append('f_total_err_{}'.format(filt))
    
    columns.append(m1)
    column_names.append('m_half_light_{}'.format(filt))
    
    columns.append(merr1)
    column_names.append('m_half_light_err_{}'.format(filt))
    
    columns.append(m2)
    column_names.append('m_total_{}'.format(filt))
    
    columns.append(merr2)
    column_names.append('m_total_err_{}'.format(filt))
    
plt.legend()

In [ ]:
petrosian_cat = detection_cat.to_table().copy()

ra, dec = detection_image.wcs.all_pix2world(petrosian_cat['maxval_xpos'], petrosian_cat['maxval_ypos'], 0)

petrosian_cat.add_column(ra*u.deg, index=1, name='ra')
petrosian_cat.add_column(dec*u.deg, index=2, name='dec')

for c, cn in zip(columns, column_names):
    petrosian_cat.add_column(c, name=cn)
petrosian_cat

In [ ]:
final_catalog_output_path = "{}_petrosian_catalog.csv".format(program_name)
print(final_catalog_output_path)

petrosian_cat.write(final_catalog_output_path, overwrite=True)

print("Done")